In [1]:
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import root_mean_squared_error
import warnings

warnings.filterwarnings("ignore")
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

sales_raw = pd.read_csv("./sales_raw.csv")
discounts_raw = pd.read_csv("./discounts_raw.csv")

sales_raw['date'] = pd.to_datetime(sales_raw['date'])
discounts_raw['date_start'] = pd.to_datetime(discounts_raw['date_start'])
discounts_raw['date_end'] = pd.to_datetime(discounts_raw['date_end'])

# Удаление дубликатов
sales_raw = sales_raw.drop_duplicates()
discounts_raw = discounts_raw.drop_duplicates()

# Функция для объединения данных по item_id и временным рамкам
def merge_sales_discounts(sales, discounts):
    # Создаем копию данных о продажах для последующего добавления информации о скидках
    merged_data = sales.copy()

    # Инициализируем новые колонки значениями NaN
    merged_data['promo_typeCode'] = np.nan
    merged_data['sale_price_before_promo'] = np.nan
    merged_data['sale_price_time_promo'] = np.nan
    
    # Для каждой строки в discounts добавляем соответствующую информацию в sales
    for i, discount in discounts.iterrows():
        mask = (sales['item_id'] == discount['item_id']) & \
               (sales['date'] >= discount['date_start']) & \
               (sales['date'] <= discount['date_end'])
        merged_data.loc[mask, 'promo_typeCode'] = discount['promo_typeCode']
        merged_data.loc[mask, 'sale_price_before_promo'] = discount['sale_price_before_promo']
        merged_data.loc[mask, 'sale_price_time_promo'] = discount['sale_price_time_promo']
    
    return merged_data

# Объединение данных
merged_data = merge_sales_discounts(sales_raw, discounts_raw)

In [2]:
merged_data

,date,item_id,qnty,promo_typeCode,sale_price_before_promo,sale_price_time_promo
0,2017-01-01,100001,7.0,NaN,NaN,NaN
1,2017-01-01,100003,2.0,NaN,NaN,NaN
2,2017-01-01,100006,13.0,NaN,NaN,NaN
3,2017-01-01,100010,5.0,NaN,NaN,NaN
4,2017-01-01,100035,53.0,NaN,NaN,NaN
...,...,...,...,...,...,...
147138,2023-12-31,100614,5.0,NaN,NaN,NaN
147139,2023-12-31,100140,4.0,6.0,749.9,689.9
147140,2023-12-31,100650,11.0,5.0,109.9,79.9
147141,2023-12-31,100182,1.0,NaN,NaN,NaN


In [3]:
# Заполнение пропусков
merged_data['promo_typeCode'].fillna(0, inplace=True)
merged_data['sale_price_before_promo'].fillna(merged_data['sale_price_before_promo'].mean(), inplace=True)
merged_data['sale_price_time_promo'].fillna(merged_data['sale_price_time_promo'].mean(), inplace=True)
merged_data['qnty'].fillna(0, inplace=True)

# Подготовка данных для ARIMA
predictions = []
items_in_december = sales_raw[sales_raw['date'].dt.month == 12]['item_id'].unique()

# Подготовка данных для оценки качества модели
all_test_actuals = []
all_test_predictions = []

for item in items_in_december:
    item_data = merged_data[merged_data['item_id'] == item].set_index('date')
    item_data = item_data[['qnty']].asfreq('D').fillna(0)
    
    # Обучение модели ARIMA
    model = ARIMA(item_data, order=(1, 1, 1))
    model_fit = model.fit()
    
    # Оценка качества модели на данных за декабрь 2023 года
    test_data = item_data['2023-12-01':'2023-12-31']
    if len(test_data) > 0:
        forecast_test = model_fit.forecast(steps=len(test_data))
        all_test_actuals.extend(test_data.values.flatten())
        all_test_predictions.extend(forecast_test.values.flatten())

# Прогноз на январь 2024 года
    forecast_dates = pd.date_range(start='2024-01-01', end='2024-01-31')
    forecast = model_fit.forecast(steps=len(forecast_dates))
    
    prediction = pd.DataFrame({
        'date': forecast_dates,
        'item_id': item,
        'prediction': forecast.values
    })
    
    predictions.append(prediction)

predictions = pd.concat(predictions)

# Сортировка предсказаний по дате и товару
predictions_sorted = predictions.sort_values(by=['date', 'item_id'])

# Сохранение результатов в файл predictions.csv
predictions_sorted.to_csv('./predictions.csv', index=False, sep=';')
print("Файл predictions.csv успешно сохранен.")

# Оценка качества модели на данных за декабрь 2023 года
rmse = root_mean_squared_error(all_test_actuals, all_test_predictions)
print(f'RMSE на данных за декабрь 2023 года: {rmse}')

Файл predictions.csv успешно сохранен.
RMSE на данных за декабрь 2023 года: 17.682225949223373
